In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from test_reconstruction import *
from scipy.fft import rfft
import pyreconstruction

In [ ]:
R=5
r=0.0
w0=26.0016
phi = 0.
snr=0.05
seed=-1
n_samples=1000
grid_size=97
N = 30

In [ ]:
def polar_conv(x, y):
    r = np.sqrt(x**2 + y**2)
    phi = np.arctan2(y,x)
    return r, phi

In [ ]:
coord = (np.arange(grid_size)+0.5)/grid_size*2*R -R
print(coord)
xx = np.array(np.meshgrid(coord,coord)).T.reshape(-1, 2)
plt.plot(xx[:,0], xx[:,1], ls='None', marker='+')
plt.show()

In [ ]:
phase_map = np.zeros(shape=(grid_size, grid_size))

for i in range(grid_size):
    for j in range(grid_size):
        x = coord[i]
        y = coord[j]
        if (x**2 + y**2)<R*R :
            r, phi = polar_conv(x, y)
            signal, noise = pyreconstruction.py_run(grid_size, n_samples, snr, seed, 1, r, phi, w0, N)
            res = np.angle(signal[j,i,438])
            phase_map[j,i] = res
            #plot_result(R, np.abs(signal[:,:,438])**2, "", False)


In [ ]:
im = plt.imshow(phase_map)
cbar = plt.colorbar(im)
cbar.ax.set_ylabel('phase')
plt.savefig('phase_map.pdf')

In [ ]:
signal, noise = pyreconstruction.py_run(grid_size, n_samples, snr, seed, 1, r, phi, w0, N)

In [ ]:
noise_c = np.abs(noise)
noise_c[noise_c==0] = np.nan
#means = np.nanmean(noise_c, axis=(1,2))
mean = np.nanmean(noise_c)
noise /= mean
signal /= mean

In [ ]:
max_vals_s = np.max(np.abs(signal)**2, axis=(0,1))
max_vals_n = np.max(np.abs(noise)**2, axis=(0,1))

In [ ]:
phases = []
for i in range(signal.shape[-1]):
    max_ind = np.unravel_index(np.argmax(np.abs(signal[:,:,i])), signal[:,:,0].shape)
    phases.append(np.angle(signal[max_ind[0], max_ind[1], i]))

phases_np = np.array(phases)
print(phases[437], phases[438])
ind = np.abs(phases_np+np.pi/2)>0.05

In [ ]:
plt.plot(max_vals_s, label='signal')
#plt.plot(max_vals_n, label='noise')
plt.xlabel('frequency')
plt.ylabel('SNR')
#plt.legend(loc='best')
#plt.savefig("frequency_snr_signal.pdf")
plt.show()
plt.show()

#max_vals_s[ind] = 0

#plt.plot(max_vals_s, label='signal')
plt.plot(max_vals_n, label='noise')
plt.xlabel('frequency')
plt.ylabel('SNR')
#plt.legend(loc='best')
#plt.savefig("frequency_snr_noise.pdf")
plt.show()
ind_max = np.argmax(max_vals_s)
ind_max_n = np.argmax(max_vals_n)
print(ind_max, ind_max_n)

In [ ]:
plt.plot(np.unwrap(np.angle(signal[50,85,:])*2)/2)
plt.show()

plt.plot(np.angle(signal[50,0,:]))
plt.show()

print(np.angle(signal[50,0,438]))

plt.plot(np.abs(signal[50,85,:]))
plt.show()

In [ ]:
plot_result(R, np.abs(signal[:,:,ind_max])**2, "wrong_f.pdf", False)
plot_result(R, np.angle(signal[:,:,ind_max]), "", False)
print(ind_max)
plot_result(R, np.abs(signal[:,:,438])**2, "correct_f.pdf", False)
plot_result(R, np.angle(signal[:,:,438]), "", False)

plot_result(R, np.abs(noise[:,:,10])**2, "just_noise.pdf", False)
plot_result(R, np.angle(noise[:,:,10]), "", False)

In [ ]:
data_freq = fft2(signal, axes=(0,1))
plt.plot(np.max(np.abs(data_freq),axis=(0,1)))
plt.show()
max_ind = np.argmax(np.abs(data_freq[0,0]))
print(max_ind)

In [ ]:
data_freq = fftshift(fft2(signal[:,:,ind_max]))

fig, ax = plt.subplots()
ax.set_aspect('equal')
im = ax.imshow(np.transpose(np.abs(data_freq)), origin='lower')
fig.colorbar(im)
plt.savefig('2ddft_noise.pdf')
plt.show()

data_freq = fftshift(fft2(signal[:,:,438]))

fig, ax = plt.subplots()
ax.set_aspect('equal')
im = ax.imshow(np.transpose(np.abs(data_freq)), origin='lower')
fig.colorbar(im)
plt.savefig('2ddft_signal.pdf')
plt.show()